# Helping Redactors to tag their content faster with IPTC tags

In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
%matplotlib inline 
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')
sys.getdefaultencoding()

- Redakteure müssen aktuell selbst schlagworte und rubriken vorschlagen.
- Das ist evtl. mühlelig und auch nicht besonders exakt

In [62]:
from IPython.display import Image
#Image(filename='tags.png') 

# Scrape Items

In [63]:
items = []
raw_data = []
for i in range(1,10):
    start = (i-1)*100
    end =  i*100
    print("start %s end %s" %(start,end))
    url = "https://www.swissinfo.ch/eng/%s/%s?view=loadMore&end=%s&begin=%s" % (start,end,end,start)
    print(url)
    results = requests.get(url)
    page_content = BeautifulSoup(results.content, "html.parser")
    for link in page_content.select("h3 a"):
        id = link["href"].split("/")[-1]
        url = "https://www.swissinfo.ch/webservice/swi-eng-2.0/deeplink?contentId=%s" % id
        print("--%s" % url)
        tmp = requests.get(url)
        r = requests.get("https://www.swissinfo.ch%s" % tmp.json()["item"]["url"]).json()
        try:
            tag = r["header"]["subjectTags"][0]["title"]
        except:
            tag = ""
        soup = BeautifulSoup(r["htmldetail"], "lxml")
        text = " ".join([text.get_text() for text in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', "p"])])
        try:
            url = r["header"]["canonical"]
        except:
            url = ""
        raw_data.append(r)
        items.append({"id": id, "url": url, "tag": tag, "text": text})

SSLError: EOF occurred in violation of protocol (_ssl.c:590)

In [42]:
items[0]

{'id': u'44623632',
 'tag': u'Society',
 'text': u'    Study examines health of Geneva\u2019s undocumented migrants - SWI swissinfo.ch             Around 2,000 long-term undocumented migrants in Geneva look set to receive official resident permits when the canton\'s "Operation Papyrus" concludes at the end of this year(\xa9 KEYSTONE / SALVATORE DI NOLFI)       Little by little, Geneva is discovering more about the thousands of undocumented immigrants thought to be in the region. The initial results of a four-year study into the health and living conditions of illegal and regularised migrants have some important early findings.\xa0Around 76,000 illegal immigrants are believed to be living in Switzerland (see infobox below). Of these, canton Geneva is home to 13,000. In 2015, the local and federal authorities launched a unique pilot scheme, known as Operation Papyrus, that looks set to grant around 2,000 long-term undocumented migrants in Geneva official resident permits when it conclude

In [48]:
from nltk.tokenize import sent_tokenize
split_items = []
for item in items:    
    sent_tokenize_list = sent_tokenize(item["text"])
    for text in sent_tokenize_list:
        split_items.append({"id": item["id"], "url": item["url"], "tag": item["tag"], "text": text.encode("utf-8")})


In [49]:
df = pd.DataFrame(split_items)
df.head()

,id,tag,text,url
0,44623632,Society,Study examines health of Geneva’s undocume...,https://www.swissinfo.ch/eng/society/living-un...
1,44623632,Society,The initial results of a four-year study into ...,https://www.swissinfo.ch/eng/society/living-un...
2,44623632,Society,"Around 76,000 illegal immigrants are believed ...",https://www.swissinfo.ch/eng/society/living-un...
3,44623632,Society,"Of these, canton Geneva is home to 13,000.",https://www.swissinfo.ch/eng/society/living-un...
4,44623632,Society,"In 2015, the local and federal authorities lau...",https://www.swissinfo.ch/eng/society/living-un...


In [50]:
len(df)

2800

# Prepare categories

In [51]:
df = pd.DataFrame(items)
df["text"] = df["text"].str#.encode('utf-8')
df["tag"] = df["tag"].str#.encode('utf-8')
df["tag"] = pd.Categorical(df["tag"])
df['code'] = df["tag"].cat.codes
df.head()

,id,tag,text,url,code
0,44623632,Society,Study examines health of Geneva’s undocume...,https://www.swissinfo.ch/eng/society/living-un...,11
1,44560650,Sci & Tech,How scientists are redefining success - SW...,https://www.swissinfo.ch/eng/sci-tech/publish-...,10
2,44623996,Politics,"COP24: Text adopted, ambitions abandoned -...",https://www.swissinfo.ch/eng/politics/opinion_...,9
3,44623032,Health,Novartis weighs reinsurance tie-up to fund...,https://www.swissinfo.ch/eng/business/reinsura...,7
4,44618048,Society,‘Being Swiss is an essential part of my id...,https://www.swissinfo.ch/eng/society/-weareswi...,11


In [59]:
dict( enumerate(df['tag'].cat.categories) )

{0: '',
 1: 'Business',
 2: 'Culture',
 3: 'Direct democracy',
 4: 'Education',
 5: 'Environment',
 6: 'Foreign Affairs',
 7: 'Health',
 8: 'Human interest',
 9: 'Politics',
 10: 'Sci & Tech',
 11: 'Society',
 12: 'Work'}

## Prepare data for training

In [53]:
import json
from pathlib import Path
import re
import random

training_data = Path("fasttext_dataset_training.txt")
test_data = Path("fasttext_dataset_test.txt")

# What percent of data to save separately as test data
percent_test_data = 0.10


def strip_formatting(string):
    string = string.lower()
    string = re.sub(r"([.!?,'/()])", r" \1 ", string)
    return string

with training_data.open("w") as train_output, \
     test_data.open("w") as test_output:

    for i,item in df.iterrows():
        #print(item)
        if (item["tag"] != "") & (item["text"] != ""):
            tag = item["code"]
            text = item["text"].replace("\n", " ")
            text = strip_formatting(text)

            fasttext_line = unicode("__label__{} {}".format(tag, text),'utf-8')

            if random.random() <= percent_test_data:
                test_output.write(fasttext_line + "\n")
            else:
                train_output.write(fasttext_line + "\n")


# Train

In [54]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model 

# Test

In [55]:
!fasttext test category_model.bin fasttext_dataset_test.txt

## Predict

In [57]:
import fasttext
import re

def strip_formatting(string):
    string = string.lower()
    string = re.sub(r"([.!?,'/()])", r" \1 ", string)
    return string

# Reviews to check
reviews = [
    "This restaurant literally changed my life. This is the best food I've ever eaten!",
    "I hate this place so much. They were mean to me.",
    "I don't know. It was ok, I guess. Not really sure what to say.",
]

# Pre-process the text of each review so it matches the training format
preprocessed_reviews = list(map(strip_formatting, reviews))

# Load the model
classifier = fasttext.load_model('reviews_model_ngrams.bin')

# Get fastText to classify each review with the model
labels, probabilities = classifier.predict(preprocessed_reviews, 1)

# Print the results
for review, label, probability in zip(reviews, labels, probabilities):
    stars = int(label[0][-1])

    print("{} ({}% confidence)".format("*" * stars, int(probability[0] * 100)))
    print(review)
    print()



ValueError: fastText: trained model cannot be opened!